# Building an ETL Pipeline

In [0]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, date, time, timedelta
import tweepy  
import numpy as np 
import pyodbc
from sqlalchemy import create_engine
import urllib
import re
import geopandas as gpd

###  Consumer and Access details

In [0]:
# Consumer:
CONSUMER_KEY    = 'uXfKji3NyWLDEGI3gXxx3tBNc'
CONSUMER_SECRET = 'eiuC2xhRfbmbY2gSplZOcL4dPSSUiqmB8SbLk6eGuZ2DaBaYQj'

# Access:
ACCESS_TOKEN  = '1219607153163603968-oOHm1E2uXjyBiBOkkKNExfAwvU4dVd'
ACCESS_SECRET = '3dZ7jyldTV1m1RxoVPv8VEOT92m2OwEPpF1Syj0j2Mu95'

In [0]:

def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1: Twitter scraper

In [0]:

def twitter_df(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_SECRET):
   
    """
    Scrapes tweets relating to "Eskom_SA" as well as the date and time the tweets were posted directly from Twitter
    
    -------------

    Arguments: 
    consumer_key
    consumer_secret 
    access_token
    access_secret 

    -------------

    Returns :
    A dataframe which containing tweets relating to Eskom as well as the date and time the tweets were posted 
    """
    
    api = twitter_setup()                                                             # creating API with athentication using the utility function we created 
    all_tweets = []                                                                   # creating an empty lost to store all the scrapes tweets 

    tweets = api.user_timeline(screen_name='ESKOM_SA',                                # pulls a maximum of 200 tweets
                           count=200,
                           include_rts = False,                         
                           tweet_mode = 'extended'
                           )
    all_tweets.extend(tweets)
  
    outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]

    df = pd.DataFrame(outtweets,columns=['Tweets','Date'])                            # creating a new dataframe to store the tweets and corresponding dates
    df.set_index('Tweets')                                                            # setting the Tweets column as the index 

    return df
twitter_df(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_SECRET)

,Tweets,Date
0,Report electricity theft anonymously to Eskom ...,2020-03-10 12:38:00
1,#POWERALERT 2\n\nDate: 10 March 2020\n\nStage ...,2020-03-10 11:50:38
2,#Chooseday: Choose to do the right thing and p...,2020-03-10 10:06:00
3,Prepaid pricing is based on usage/consumption ...,2020-03-10 08:30:00
4,"@TSIKALIRA Hi, pls DM us your contact details ...",2020-03-10 06:32:50
5,#POWERALERT 1\nDate: 10 March 2020\n\nStage 2 ...,2020-03-10 05:37:17
6,Protect your electrical appliances by unpluggi...,2020-03-10 04:30:00
7,@Jeandre_S @CityTshwane @SABCNewsOnline @SABCR...,2020-03-09 17:32:41
8,@Antifreez777za @SikonathiM Please see our tim...,2020-03-09 17:26:55
9,@Zwanedefault Work of scam artists,2020-03-09 17:25:20


In [0]:
def extract_municipality_hashtags(df): 
    """
    This function extracts the municpality and hashtags from tweets 
    and inserts them into the twitter dataframe under the columns 'municipality' amd 'hashtags'.

    -----------
    Arguments:
    dataframe containing tweets relating to Eskom_SA as well as the date and time that the tweet was posted

    -----------
    Returns:
    dataframe containing tweets relating to Eskom_SA, the date and time that the tweet was posted as well as the corresponding municipality and hashtags 
    """
    
    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    handles = list(mun_dict.keys())
    
    df['municipality'] = df['Tweets'].str.extract('({})'.format('|'.join(handles)), expand=False).fillna(np.nan)        #Finding all the handles in the tweet strings if there's not one, we just insert a NaN val into the df
    df['municipality'] = df['municipality'].map(mun_dict)                                                               #Replacing all the handles with the city names.
    
    df['hashtags'] = df['Tweets'].str.findall(r'#.*?(?=\s|$)')                                                          #Using a reg ex string to find all sub-strings that matches a hastag
    df['hashtags'] = df['hashtags'].apply(lambda x: np.nan if len(x)==0 else [x.lower() for x in x])                    #Converting all the extracted hastags to lowercase or NaN if there's not one in the tweet string, then adding it hastag column of the df
    
    return df
extract_municipality_hashtags(twitter_df(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_SECRET))

,Tweets,Date,municipality,hashtags
0,Report electricity theft anonymously to Eskom ...,2020-03-10 12:38:00,NaN,NaN
1,#POWERALERT 2\n\nDate: 10 March 2020\n\nStage ...,2020-03-10 11:50:38,NaN,[#poweralert]
2,#Chooseday: Choose to do the right thing and p...,2020-03-10 10:06:00,NaN,[#chooseday:]
3,Prepaid pricing is based on usage/consumption ...,2020-03-10 08:30:00,NaN,[#useelectricitysmartly]
4,"@TSIKALIRA Hi, pls DM us your contact details ...",2020-03-10 06:32:50,NaN,NaN
5,#POWERALERT 1\nDate: 10 March 2020\n\nStage 2 ...,2020-03-10 05:37:17,NaN,[#poweralert]
6,Protect your electrical appliances by unpluggi...,2020-03-10 04:30:00,NaN,[#loadshedding]
7,@Jeandre_S @CityTshwane @SABCNewsOnline @SABCR...,2020-03-09 17:32:41,Tshwane,NaN
8,@Antifreez777za @SikonathiM Please see our tim...,2020-03-09 17:26:55,NaN,NaN
9,@Zwanedefault Work of scam artists,2020-03-09 17:25:20,NaN,NaN


### Function 3: Updating SQL Database with pyODBC

In [0]:
def pyodbc_twitter(connection, df, twitter_table):
    """
    This function updates the existing twitter table in the SQL database 
    by replacing the existing contents with the latest 200 tweets scraped directly from twitter
    
    ------------
    Arguments:
    connection
    dataframe 
    SQL table 
    
    ------------
    Returns:
    This function does not return anything, it updates a table in SQL server 
    """
    
    params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=MANDLA63\SQLEXPRESS;DATABASE=gather_eskom;UID=sa;trusted_connection='true'")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    df.to_sql('twitter_table', con=engine, if_exists='replace')
    
    #Initializing connection, update  and store twitter_table on SQL
    conn = pyodbc.connect(driver='{SQL Server}' ,
                     host='MANDLA63\SQLEXPRESS' ,
                     database='gather_eskom' ,
                     trusted_connection='true',
                      user='sa')
    
    twitter_table= pd.read_sql_query('select * from dbo.twitter_table',conn)

,index,Tweets,Date,municipality,hashtags
0,0,Report electricity theft anonymously to Eskom ...,2020-03-10 12:38:00,None,nan
1,1,#POWERALERT 2\n\nDate: 10 March 2020\n\nStage ...,2020-03-10 11:50:38,None,['#poweralert']
2,2,#Chooseday: Choose to do the right thing and p...,2020-03-10 10:06:00,None,['#chooseday:']
3,3,Prepaid pricing is based on usage/consumption ...,2020-03-10 08:30:00,None,['#useelectricitysmartly']
4,4,"@TSIKALIRA Hi, pls DM us your contact details ...",2020-03-10 06:32:50,None,nan


### Extra Eskom data